# Deep Learning Series


<span style="color:gray">
<br>

1. Neural Net - part.1

</span>

2. <b>Convolution Neural Network</b>


<span style="color:gray">

3. Neural Net - part.2


4. Recursive Nerural Network
</span>

## What is Convolutional Neural Network ?

Convolution은 '합성곱'으로 . 따라서 CNN은 합성곱을 하는 신경망이다.

Convolution Layer의 특징은 데이터의 형상을 유지해준다는 점이다.

이전에 사용한 Linear Layer(full connected layer)의 경우, 어떤 차원의 데이터든 1차원 데이터로 평탄화하기 때문에 데이터의 형상이 무시되었다. 

하지만 Convolution Layer의 경우 3차원으로 입력 받은 데이터를 3차원으로 출력할 수 있기 때문에 데이터 형상을 유지할 수 있다.

따라서 CNN이 이미지 분석에 강한 이유도, 이미지 데이터에는 RGB 값이나 픽셀 거리에 따른 연관성 여부 등 다양한 정보가 고차원으로 담겨있는데 Linear Layer는 이를 1차원으로 압축해버리기 때문에 상대적으로 Convolution Layer에 비해 정보의 손실이 많이 발생하기 때문이다.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [2]:
from tqdm import tqdm_notebook

In [3]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(),
                        target_transform=None, download = True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(),
                        target_transform=None, download = True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                          shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                          shuffle=False, num_workers=2, drop_last=True)

## Architecture of Convolution Neural Network

<img src="img/CNN_05.PNG">

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 5),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

## Convolution Layer
<img src="img/CNN_01.PNG">

다음의 이미지를 참고해서 설명을 진행하겠다.

<img src="img/CNN_04.PNG">

고먐미의 저주에 걸린 고양이를 분류하는 모델을 만든다고 하자.

고먐미는 고양이에 비해 직각으로 각진 부분이 많다. 
<br>따라서 다음 예시와 같이 생긴 필터는 고양이보다 고먐미 이미지 데이터에서 더 합성곱의 값을 출력할 것이다.

이런 방식으로 feature map을 만들어서 분류 모델을 학습한다.

인자에 대한 설명은 아래와 같다.



* in_channels 
    - 현재 input한 데이터의 채널 수이다.


* out_channels  
    - 필터를 적용해서 만든 데이터의 개수 (여기선 필터 16개를 한 데이터에 적용해서 16개의 결과물을 만들어내었다.)
    - <span style="color:red"><b>??? 근데 channel을 줄일 떄는 어떻게하지???? 하나의 채널로 합친다음 다시 채널을 나누는건가</b>....밑바닥부터시작하는딥러닝 p.234 참고 </span>
    

* kernel_size
    - 커널 사이즈는 필터의 크기로 만일 2로 설정된다면 2x2 사이즈의 필터가 생성된다.


<img src="img/CNN_03.PNG">

* stride
    - 출력크기를 조정할 목적으로 사용된다.
    - 필터를 적용하는 위치의 간격으로, 스트라이드의 수만큼 필터가 이동하게 된다.


* padding 
    - 출력 크기를 조정할 목적으로 사용된다.
    - 입력 데이터 주변을 특정 값(예컨대 0)으로 채우는 방법이 있다.|

Convolution leverages three important ideas that can help improve a machine learning system

1. Sparse interactions

2. Parameter sharing

3. Equivariant representations

1. Sparse interactions

<img src="img/CNN_06.PNG">

2. Parameter sharing

<img src="img/CNN_07.PNG">

3. Equivariant representations



## Pooling Layer
<img src="img/CNN_02.PNG">

* Pooling Layer ?
    - pooling은 데이터 크기를 조정할 목적으로 사용된다. (세로/가로 방향의 공간을 줄이는 연산)
    - 종류에는 Max pooling, Average pooling 등이 있다.



* Pooling Layer의 특징
    - 채널 수에 영향을 주지 않는다.
    - 입력의 변화에 영향을 적게 받는다. (Robust하다)

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
loss_arr = []
for i in tqdm_notebook(range(num_epoch)):
    for j,[image, label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j%1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3067, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2646, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1282, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0822, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0893, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1050, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0768, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0474, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0347, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0276, device='cuda:0', grad_fn=<NllLossBackward>)



In [15]:
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y_= label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
    
    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 98.53765869140625
